# IDLE EDGE

In [ ]:
import cv2
import numpy as np
import scipy.ndimage as nd
from scipy import signal
import matplotlib.pyplot as plt

# Read the image
img = cv2.imread(r"C:\Users\Moulishwaran\OneDrive\Pictures\U23PG507DTS021.jpg")
original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_gaussian = cv2.GaussianBlur(gray, (3, 3), 0)

# Canny
img_canny = cv2.Canny(img, 100, 200)

# Sobel
img_sobelx = cv2.Sobel(img_gaussian, cv2.CV_8U, 1, 0, ksize=5)
img_sobely = cv2.Sobel(img_gaussian, cv2.CV_8U, 0, 1, ksize=5)
img_sobel = img_sobelx + img_sobely

# Prewitt
kernelx = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
kernely = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
img_prewittx = cv2.filter2D(img_gaussian, -1, kernelx)
img_prewitty = cv2.filter2D(img_gaussian, -1, kernely)
img_prewitt = img_prewittx + img_prewitty

# Laplacian (Second Order Derivation)
laplacian = cv2.Laplacian(img_gaussian, cv2.CV_64F)

# LoG (Laplacian of Gaussian)
LoG = nd.gaussian_laplace(img_gaussian, 2)
thres = np.absolute(LoG).mean() * 0.75
output = np.zeros(LoG.shape)
w = output.shape[1]
h = output.shape[0]

for y in range(1, h - 1):
    for x in range(1, w - 1):
        patch = LoG[y-1:y+2, x-1:x+2]
        p = LoG[y, x]
        maxP = patch.max()
        minP = patch.min()
        if (p > 0):
            zeroCross = True if minP < 0 else False
        else:
            zeroCross = True if maxP > 0 else False
        if ((maxP - minP) > thres) and zeroCross:
            output[y, x] = 1

# DoG (Difference of Gaussian)
A = np.array([[0, 0, -1, -1, -1, 0, 0],
              [0, -2, -3, -3, -3, -2, 0],
              [-1, -3, 5, 5, 5, -3, -1],
              [-1, -3, 5, 16, 5, -3, -1],
              [-1, -3, 5, 5, 5, -3, -1],
              [0, -2, -3, -3, -3, -2, 0],
              [0, 0, -1, -1, -1, 0, 0]],
             dtype=np.float32)

img2 = cv2.imread(r"C:\Users\Moulishwaran\OneDrive\Pictures\U23PG507DTS021.jpg", 1)
ratio = img2.shape[0] / 500.0
new_width = int(img2.shape[1] / ratio)
original = img2.copy()
nimg = cv2.resize(img2, (new_width, 500))

I1 = signal.convolve2d(nimg[:, :, 0], A)
I1 = np.absolute(I1)
I1 = (I1 - np.min(I1)) / float(np.max(I1) - np.min(I1))

I2 = cv2.GaussianBlur(I1, (5, 5), 0)

# Plot the results
plt.subplot(2, 4, 1), plt.imshow(original_img)
plt.title('Original Image'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 2), plt.imshow(gray, cmap='gray')
plt.title('Gray Scale Image'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 3), plt.imshow(img_canny, cmap='gray')
plt.title('Canny Edge Detection'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 4), plt.imshow(img_sobel, cmap='gray')
plt.title('Sobel Edge Detection'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 5), plt.imshow(img_prewitt, cmap='gray')
plt.title('Prewitt Edge Detection'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 6), plt.imshow(laplacian, cmap='gray')
plt.title('Laplacian Edge Detection'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 7), plt.imshow(output, cmap='gray')
plt.title('LoG Edge Detection'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 8), plt.imshow(I2, cmap='gray')
plt.title('DoG Edge Detection'), plt.xticks([]), plt.yticks([])

plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()


# COLAB

# MATLAB EDGE 

# MATLAB 

In [ ]:
% Read the image
img = imread('C:\Users\Moulishwaran\OneDrive\Pictures\03-02-2024 18_59_28.png');
original_img = img;

% Convert to gray scale
gray = rgb2gray(img);

% Gaussian blur
img_gaussian = imgaussfilt(gray, 1);

% Canny edge detection
img_canny = edge(gray, 'Canny');

% Sobel edge detection
img_sobelx = imfilter(double(img_gaussian), fspecial('sobel')');
img_sobely = imfilter(double(img_gaussian), fspecial('sobel'));
img_sobel = sqrt(img_sobelx.^2 + img_sobely.^2);

% Prewitt edge detection
img_prewittx = imfilter(double(img_gaussian), fspecial('prewitt')');
img_prewitty = imfilter(double(img_gaussian), fspecial('prewitt'));
img_prewitt = sqrt(img_prewittx.^2 + img_prewitty.^2);

% Laplacian edge detection
laplacian = imfilter(double(img_gaussian), fspecial('laplacian'));

% LoG (Laplacian of Gaussian)
LoG = imfilter(double(img_gaussian), fspecial('log', 5, 0.5));
thres = mean2(abs(LoG)) * 0.75;
output = zeros(size(LoG));
for y = 2:size(LoG, 1)-1
    for x = 2:size(LoG, 2)-1
        patch = LoG(y-1:y+1, x-1:x+1);
        p = LoG(y, x);
        maxP = max(patch(:));
        minP = min(patch(:));
        if p > 0
            zeroCross = minP < 0;
        else
            zeroCross = maxP > 0;
        end
        if (maxP - minP) > thres && zeroCross
            output(y, x) = 1;
        end
    end
end

% DoG (Difference of Gaussian)
A = [0 0 -1 -1 -1 0 0;
     0 -2 -3 -3 -3 -2 0;
     -1 -3 5 5 5 -3 -1;
     -1 -3 5 16 5 -3 -1;
     -1 -3 5 5 5 -3 -1;
     0 -2 -3 -3 -3 -2 0;
     0 0 -1 -1 -1 0 0];
nimg = imresize(img, [500 NaN]);
I1 = conv2(double(nimg(:, :, 1)), A, 'same');
I1 = abs(I1);
I1 = (I1 - min(I1(:))) / (max(I1(:)) - min(I1(:)));
I2 = imgaussfilt(I1, 1);

% Display results
figure;
subplot(2, 4, 1), imshow(original_img), title('Original Image');
subplot(2, 4, 2), imshow(gray), title('Gray Scale Image');
subplot(2, 4, 3), imshow(img_canny), title('Canny Edge Detection');
subplot(2, 4, 4), imshow(img_sobel, []), title('Sobel Edge Detection');
subplot(2, 4, 5), imshow(img_prewitt, []), title('Prewitt Edge Detection');
subplot(2, 4, 6), imshow(laplacian, []), title('Laplacian Edge Detection');
subplot(2, 4, 7), imshow(output, []), title('LoG Edge Detection');
subplot(2, 4, 8), imshow(I2, []), title('DoG Edge Detection');


# IDLE MORPH

In [ ]:
import cv2 as cv
import numpy as np
img = cv.imread(r"C:\Users\Moulishwaran\OneDrive\Pictures\U23PG507DTS021.jpg")
kernel = np.ones((5,5),np.uint8)
cv.imshow('original',img)
cv.imwrite('original_lena.png',img)

erosion = cv.erode(img,kernel,iterations = 1)

resized_erosion = cv.resize(erosion, (300, 300), interpolation=cv.INTER_AREA)
cv.imshow('Resized Erosion Image', resized_erosion)
cv.imwrite('resized_erosion_lena.png', resized_erosion)

dilation = cv.dilate(img,kernel,(300,300),iterations = 1)
cv.imshow('dilation image', dilation)
cv.imwrite('dilation.png',dilation)

resized_dilation = cv.resize(dilation, (300, 300), interpolation=cv.INTER_AREA)
cv.imshow('Resized dilation Image', resized_dilation)
cv.imwrite('resized_dilation_lena.png', resized_dilation)

opening = cv.morphologyEx(img, cv.MORPH_OPEN, kernel)
cv.imshow('Open operation image', opening)
cv.imwrite('opening_lena.png',opening)

closing = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)
cv.imshow('Close operation image', closing)
cv.imwrite('closing_lena.png',closing)

gradient = cv.morphologyEx(img, cv.MORPH_GRADIENT, kernel)
cv.imshow('Gradient operation image', gradient)
cv.imwrite('gradient_lena.png',gradient)


tophat = cv.morphologyEx(img, cv.MORPH_TOPHAT, kernel)
cv.imshow('Top Hat operation image', tophat)
cv.imwrite('tophat_lena.png',tophat)

blackhat = cv.morphologyEx(img, cv.MORPH_BLACKHAT, kernel)
cv.imshow('Black Hat operation image', blackhat)
cv.imwrite('blackhat_lena.png',blackhat)


rectangle = cv.morphologyEx(img, cv.MORPH_RECT, kernel)
cv.imshow('Rectangular operation image', rectangle)
cv.imwrite('rectangle_lena.png',rectangle)

ellipse = cv.morphologyEx(img, cv.MORPH_ELLIPSE, kernel)
cv.imshow('Ellipse operation image', ellipse)
cv.imwrite('ellipse_lena.png',ellipse)


cross = cv.morphologyEx(img, cv.MORPH_CROSS, kernel)
cv.imshow('Cross operation image', cross)
cv.imwrite('cross_lena.png',cross)

cv.waitKey(0)
cv.destroyAllWindows()


# MATLAB MORPH

In [ ]:
% Read the image
img = imread('C:\Users\Moulishwaran\OneDrive\Pictures\03-02-2024 18_59_28.png');

% Convert to grayscale if it's not already
if size(img, 3) == 3
    img = rgb2gray(img);
end

% Erosion
se = strel('square', 5);
erosion = imerode(img, se);
figure; imshow(erosion); title('Eroded Image');

% Dilation
dilation = imdilate(img, se);
figure; imshow(dilation); title('Dilated Image');

% Opening
opening = imopen(img, se);
figure; imshow(opening); title('Opening');

% Closing
closing = imclose(img, se);
figure; imshow(closing); title('Closing');

% Top-hat (White top-hat)
tophat = imtophat(img, se);
figure; imshow(tophat); title('Tophat');

% Black-hat
blackhat = imblackhat(img, se);
figure; imshow(blackhat); title('Blackhat');




# IDLE HIST

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('Lenna_(test_image).png',0)
if img is None:
    print('Could not open or find the image:', img)
    exit(0)          
equ = cv2.equalizeHist(img)
res = np.hstack((img,equ)) #stacking images side-by-side
#cv2.imwrite('res.png',res)
cv2.imshow('resultant image', res)

hist,bins = np.histogram(img.flatten(),256,[0,256])
cdf = hist.cumsum()
cdf_normalized = cdf * hist.max()/ cdf.max()

plt.plot(cdf_normalized, color = 'b')
plt.hist(img.flatten(),256,[0,256], color = 'r')
plt.xlim([0,256])
plt.legend(('cdf','histogram'), loc = 'upper left')
plt.show()


cv2.waitKey(0)
cv2.destroyAllWindows()



# MATLAB HIST

In [ ]:
% Read the image
image = imread("C:\Users\Moulishwaran\OneDrive\Pictures\cat.jpeg");

if isempty(image)
    disp('Error: Unable to read the image file.');
else
    % Convert the image to grayscale
    gray_image = rgb2gray(image);

    % Prepare images and titles
    images = {gray_image, image};
    titles = {'Grayscale Image', 'Color Image'};
    channels = {'Grayscale', 'Blue', 'Green', 'Red'};
    colors = {'k', 'b', 'g', 'r'};

    figure('Position', [100, 100, 1200, 600]);

    % Display images
    for i = 1:2
        subplot(2, 3, i);
        if i == 1
            imshow(images{i});
            colormap('gray');
        else
            imshow(images{i});
        end
        title(titles{i});
        axis off;
    end

    % Display histograms
    for j = 1:length(channels)
        subplot(2, 3, j+2);
        if j == 1
            hist_data = imhist(gray_image);
        else
            hist_data = imhist(image(:,:,j-1));
        end
        plot(hist_data, 'Color', colors{j});
        title([channels{j}, ' Histogram']);
        xlabel('Bins');
        ylabel('# of Pixels');
        xlim([0, 256]);
    end

    tight_layout();
end


# IDLE WATERSHED

In [ ]:
from __future__ import print_function
import numpy as np
import cv2 as cv

class Sketcher:
    def __init__(self, windowname, dests, colors_func):
        self.prev_pt = None
        self.windowname = windowname
        self.dests = dests
        self.colors_func = colors_func
        self.dirty = False
        self.show()

        cv.setMouseCallback(self.windowname, self.on_mouse)

    def show(self):
        cv.imshow(self.windowname, self.dests[0])

    def on_mouse(self, event, x, y, flags, param):
        if event == cv.EVENT_LBUTTONDOWN:
            self.prev_pt = (x, y)
        elif event == cv.EVENT_LBUTTONUP:
            self.prev_pt = None
        elif event == cv.EVENT_MOUSEMOVE and self.prev_pt:
            pt = (x, y)
            for dst, color in zip(self.dests, self.colors_func()):
                cv.line(dst, self.prev_pt, pt, color, 5)
            self.prev_pt = pt
            self.dirty = True
            self.show()

class App:
    def __init__(self, fn):
        self.img = cv.imread(fn)
        if self.img is None:
            raise Exception('Failed to load image file: %s' % fn)

        h, w = self.img.shape[:2]
        self.markers = np.zeros((h, w), np.int32)
        self.markers_vis = self.img.copy()
        self.cur_marker = 1
        self.colors = np.int32( list(np.ndindex(2, 2, 2)) ) * 255

        self.auto_update = True
        self.sketch = Sketcher('img', [self.markers_vis, self.markers], self.get_colors)

    def get_colors(self):
        return list(map(int, self.colors[self.cur_marker])), self.cur_marker

    def watershed(self):
        m = self.markers.copy()
        cv.watershed(self.img, m)
        overlay = self.colors[np.maximum(m, 0)]
        vis = cv.addWeighted(self.img, 0.5, overlay, 0.5, 0.0, dtype=cv.CV_8UC3)
        cv.imshow('watershed', vis)

    def run(self):
        while cv.getWindowProperty('img', 0) != -1 or cv.getWindowProperty('watershed', 0) != -1:
            ch = cv.waitKey(50)
            if ch == 27:
                break
            if ch >= ord('1') and ch <= ord('7'):
                self.cur_marker = ch - ord('0')
                print('marker: ', self.cur_marker)
            if ch == ord(' ') or (self.sketch.dirty and self.auto_update):
                self.watershed()
                self.sketch.dirty = False
            if ch in [ord('a'), ord('A')]:
                self.auto_update = not self.auto_update
                print('auto_update is', ['off', 'on'][self.auto_update])
            if ch in [ord('r'), ord('R')]:
                self.markers[:] = 0
                self.markers_vis[:] = self.img
                self.sketch.show()
        cv.destroyAllWindows()

if __name__ == '__main__':
    print(__doc__)
    import sys
    try:
        fn = sys.argv[1]
    except:
        fn = r'C:\Users\Moulishwaran\OneDrive\Pictures\cat.jpeg'
    App(cv.samples.findFile(fn)).run()


# idle hough line 

In [ ]:
import cv2
import numpy as np

img = cv2.imread(r"C:\Users\Moulishwaran\OneDrive\Pictures\cat.jpeg")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)


edges = cv2.Canny(gray,100,200,apertureSize = 3)
cv2.imshow('edges',edges)
cv2.waitKey(0)

minLineLength = 30
maxLineGap = 10
lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
for x1,y1,x2,y2 in lines[0]:
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv2.imshow('hough',img)
cv2.waitKey(0)


# MATLAB 

In [ ]:
% Read the image
img = imread('C:\Users\Moulishwaran\OneDrive\Pictures\cat.jpeg');

% Convert the image to grayscale
gray = rgb2gray(img);

% Perform Canny edge detection
edges = edge(gray, 'Canny', [0.1 0.2]);

% Display the edge-detected image
imshow(edges);
title('Edges');

% Parameters for Hough transform
minLineLength = 30;
maxLineGap = 10;

% Perform Hough line detection
[H, theta, rho] = hough(edges);
peaks = houghpeaks(H, 100, 'threshold', ceil(0.3 * max(H(:))));
lines = houghlines(edges, theta, rho, peaks, 'FillGap', maxLineGap, 'MinLength', minLineLength);

% Draw the lines on the original image
figure, imshow(img), hold on
for k = 1:length(lines)
    xy = [lines(k).point1; lines(k).point2];
    plot(xy(:,1), xy(:,2), 'LineWidth', 2, 'Color', 'green');
end

title('Hough Lines');
hold off;
